In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-18 00:23:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.18MB/s    in 0.2s    

2021-11-18 00:23:57 (5.18 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Grocery").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True,inferSchema=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

2402458
2402212
2402212


In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Review id table 
review_id_tabledf=df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_tabledf.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100WFYHG5LBQZ|   46873148|B003PWC346|     822134010| 2014-11-06|
|R10174PSJAET60|   29532905|B00PT66JYQ|     749978411| 2015-01-09|
|R101DUK4KGSZKO|   47594902|B000REGVC0|     245638153| 2008-02-18|
|R102BWOXH8BAGB|   21812686|B00M2OGS08|     715217277| 2015-02-25|
|R102FRHV3HVRYH|   20091223|B004EKHMXU|     783192559| 2014-05-26|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.types import *

In [ ]:
review_id_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
review_id_tabledf = review_id_tabledf.withColumn("customer_id",review_id_tabledf["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_tabledf["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_tabledf["review_date"].cast(DateType()))

In [ ]:
review_id_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
print(review_id_tabledf.count())

2402212


In [ ]:
review_id_tabledf = review_id_tabledf.dropDuplicates(["review_id"])

In [ ]:
# product table 
products_tabledf=df.select(["product_id", "product_title"])
products_tabledf.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003PWC346|Frontier CO-OP Ce...|
|B00PT66JYQ|Morning Pep Pure ...|
|B000REGVC0|Kellogg's Bite Si...|
|B00M2OGS08|Surge Citrus Flav...|
|B004EKHMXU|Healthworks Cacao...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
products_tabledf.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# customer table 
customer_tabledf=df.select(["customer_id"])
customer_tabledf.show(5)

+-----------+
|customer_id|
+-----------+
|   46873148|
|   29532905|
|   47594902|
|   21812686|
|   20091223|
+-----------+
only showing top 5 rows



In [ ]:
customer_tabledf = df.groupBy('customer_id').agg({"customer_id": "count"})
customer_tabledf.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   15247147|                 1|
|   10069897|                 1|
|   16127586|                 3|
|    1688219|                 1|
|    4578543|                 1|
+-----------+------------------+
only showing top 5 rows



In [ ]:
customer_tabledf = customer_tabledf.withColumnRenamed("count(customer_id)", "customer_count")
customer_tabledf.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15247147|             1|
|   10069897|             1|
|   16127586|             3|
|    1688219|             1|
|    4578543|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
customer_tabledf.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
customer_tabledf = customer_tabledf.withColumn("customer_id",customer_tabledf["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customer_tabledf["customer_count"].cast(IntegerType()))

In [ ]:
customer_tabledf.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
# vine table
vine_tabledf=df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
vine_tabledf.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100WFYHG5LBQZ|          5|            0|          0|   N|
|R10174PSJAET60|          4|           14|         16|   N|
|R101DUK4KGSZKO|          5|            1|          1|   N|
|R102BWOXH8BAGB|          5|            0|          0|   N|
|R102FRHV3HVRYH|          1|            2|          2|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
vine_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
vine_tabledf = vine_tabledf.withColumn("star_rating",vine_tabledf["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_tabledf["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_tabledf["total_votes"].cast(IntegerType()))

In [ ]:
vine_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Postgres Setup

In [1]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1-instance-1.cnfb0rdvmbzb.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [2]:
# Write DataFrame to active_user table in RDS
review_id_tabledf.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_tabledf.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customer_tabledf.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_tabledf.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)